In [40]:
import numpy as np
import torch
from itertools import product
from scipy import stats

In [90]:
# Linear regression
num_samples = 100
num_features = 700
num_groups = 3

np.random.seed(42)
noise_level = 0.1
true_beta = np.random.randn( num_features, 1 ) # does not include the identifier weight.


In [209]:
def fully_structured_covariance(d):
    return np.diag( np.random.uniform(0.3, 4, size = num_features) ** 2 )

def identity_covariance(d):
    return np.identity( d )

class Group:

    ID = 0

    def __init__(self, covariance_generator = lambda: identity_covariance(num_features) ):

        self.id = Group.ID + 1
        Group.ID += 1

        self.cov = covariance_generator()
        self.x_dist = stats.multivariate_normal( cov = self.cov )

        self.x = None
        self.y = None
    
    def _generate_x(self, n_samples, ID = False , **kwargs):
        x = self.x_dist.rvs(size= n_samples, **kwargs)
        if ID:
            identifier = np.repeat( self.ID , repeats=n_samples)
            x = np.column_stack( (identifier, x))
        return x
    
    def generate(self, n_samples, beta = true_beta):
        x = self._generate_x(num_samples)
        y = np.dot(x, beta) + noise_level * np.random.randn(num_samples, 1)
        self.data = [x, y]
    
    @property
    def data(self):
        return self.x, self.y
    
    @data.setter
    def data(self, value):
        x, y = value # unpack
        self.x = torch.tensor(x, dtype = torch.float32, requires_grad=False)
        self.y = torch.tensor(y, dtype = torch.float32, requires_grad=False)




# Create the data

cov_generator = lambda: fully_structured_covariance(num_features)
groups = [ Group(cov_generator) for _ in range(num_groups) ]

Data = {}
for group in groups:
    group.generate(num_samples, beta= true_beta)


X = torch.cat( [ group.x for group in groups ], dim = 0 )
Y = torch.cat( [ group.y for group in groups ], dim = 0 )

In [210]:

class LinearModel(torch.nn.Module):

    def __init__(self, input_dimension, **kwargs) -> None:
        super(LinearModel, self).__init__()
        # Initialize at zero. 
        self.linear = torch.nn.Linear( input_dimension, 1, bias = False, **kwargs )
        self._initialize(0.0)
    
    @torch.no_grad()
    def _initialize(self, value):
        self.linear.weight.fill_(value)
        if self.linear.weight.grad is not None:
            self.linear.weight.grad.detach_()
            self.linear.weight.grad.zero_()
    

    def forward(self, x):
        out = self.linear(x)
        return out
    
    def risk(self, x, y):
        loss = torch.nn.MSELoss()
        mse = loss( self(x) , y )
        return mse
    
    
def optimize_GD(model, regularization = 0.1, max_iter = 5000, lr = 1e-3, weight_decay = 0.0):

    lmbd = torch.tensor([regularization], requires_grad=False)
    optimizer = torch.optim.SGD( params = model.parameters(), lr = lr, weight_decay=weight_decay )

    flag = True
    iteration = 0
    while flag and iteration < max_iter:
        optimizer.zero_grad()

        standard_risk = model.risk( X, Y )
        adversarial_risk = torch.max( torch.stack( [ model.risk(group.x, group.y) for group in groups ] ) )
        objective = lmbd * standard_risk + adversarial_risk

        objective.backward()

        optimizer.step()

        if iteration % 100 == 0:
            print(f" Objective loss is {objective.item():.4f}  = {lmbd.item():.2f} * { standard_risk.item():.4f} + {adversarial_risk.item():.4f}")
        
        iteration += 1

        # Check convergence
        if objective.item() < 1e-4:
            flag = False
    
    if iteration == max_iter:
        print("Maximum iteration reached.")




In [211]:
# Solve the weighted regression problem using vanilla GD
lmbd = torch.tensor([0.1], requires_grad=False)
max_iter = 2000

model = LinearModel(num_features)
optimize_GD(model, regularization= 0.5, max_iter= max_iter)


 Objective loss is 9899.7217  = 0.50 * 6259.2856 + 6770.0786
 Objective loss is 37.7625  = 0.50 * 23.8024 + 25.8613
 Objective loss is 4.5622  = 0.50 * 2.9503 + 3.0870
 Objective loss is 0.9338  = 0.50 * 0.5847 + 0.6415
 Objective loss is 0.2311  = 0.50 * 0.1464 + 0.1579
 Objective loss is 0.0671  = 0.50 * 0.0421 + 0.0461
 Objective loss is 0.0200  = 0.50 * 0.0129 + 0.0135
 Objective loss is 0.0063  = 0.50 * 0.0041 + 0.0042
 Objective loss is 0.0021  = 0.50 * 0.0014 + 0.0015
 Objective loss is 0.0007  = 0.50 * 0.0005 + 0.0005
 Objective loss is 0.0002  = 0.50 * 0.0002 + 0.0002


In [205]:
# OLS
ols_beta = torch.linalg.lstsq( X, Y ).solution

# Subgroups- OLS
subgroup_beta = [ torch.linalg.lstsq( group.x, group.y ).solution for group in groups ]

In [206]:
import pandas as pd
import matplotlib.pyplot as plt

betas = np.hstack( [model.linear.weight.data.numpy().T, ols_beta.data.numpy()] + [beta.data.numpy() for beta in subgroup_beta ] )
betas = pd.DataFrame( betas, columns=["model", "OLS"] + [f"Group {i} OLS" for i in range(num_groups)] )



In [207]:
betas.corr()

,model,OLS,Group 0 OLS,Group 1 OLS,Group 2 OLS
model,1.000000,1.000000,0.536850,0.529683,0.652408
OLS,1.000000,1.000000,0.536831,0.529662,0.652376
Group 0 OLS,0.536850,0.536831,1.000000,0.104204,0.178770
Group 1 OLS,0.529683,0.529662,0.104204,1.000000,0.185673
Group 2 OLS,0.652408,0.652376,0.178770,0.185673,1.000000
